<a href="https://colab.research.google.com/github/yuta-kj/portfolio_for_job_hunting/blob/master/zillow_prize_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/kaggle/zillow_prize/

/content/gdrive/MyDrive/kaggle/zillow_prize


In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.2 MB/s 
     |████████████████████████████████| 210 kB 60.5 MB/s 
     |████████████████████████████████| 81 kB 10.4 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 150 kB 51.9 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 113 kB 47.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=567c474959f02b23a0e0a6bf87a487af85bbe3734be1ab2945a8b5cee5cccdbf
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
print('Loading Properties ...')
properties2016 = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

train2016 = add_date_features(train2016)
train2017 = add_date_features(train2017)

print('Loading Sample ...')
sample_submission = pd.read_csv('../input/sample_submission.csv', low_memory = False)

print('Merge Train with Properties ...')
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print('Tax Features 2017  ...')
train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan

print('Concat Train 2016 & 2017 ...')
train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(sample_submission[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
train_2016_df_ = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/train_2016_v2.csv", parse_dates=["transactiondate"], index_col='parcelid')
train_2017_df_ = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/train_2017.csv", parse_dates=["transactiondate"], index_col='parcelid')
prop_2016_df = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/properties_2016.csv", index_col='parcelid', low_memory=False)
prop_2017_df = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/properties_2017.csv", index_col='parcelid', low_memory=False)
sample_sub = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/sample_submission.csv", index_col='parceid')


# Merge datasets
train_2016_df = pd.merge(train_2016_df_, prop_2016_df, on='parcelid', how='left')
train_2017_df = pd.merge(train_2017_df_, prop_2017_df, on='parcelid', how='left')

X_full = pd.concat([train_2016_df, train_2017_df], axis=0)

In [ ]:
# float64->float32にしてメモリを効率化
import numpy as np
for c, dtype in zip(X_full.columns, X_full.dtypes):
	if dtype == np.float64:
		X_full[c] = X_full[c].astype(np.float32)

In [ ]:
sample_sub = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/sample_submission.csv")
sample_sub.set_index('ParcelId', inplace=True)
sample_sub.rename(columns={'ParcelId': 'parcelid'}, inplace=True)

In [ ]:
# 目的変数の欠損値の除去
X_full.dropna(axis=0, subset=['logerror'], inplace=True)
y = X_full.logerror
X_full.drop(['logerror'], axis=1, inplace=True)

In [ ]:
# Break off validation set from training data
X_tr_fl, X_val_fl, y_tr, y_val = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
categorical_cols = [cname for cname in X_tr_fl.columns if
                    X_tr_fl[cname].nunique() < 10 and 
                    X_tr_fl[cname].dtype == "object"]

In [ ]:
# Select numerical columns
numerical_cols = [cname for cname in X_tr_fl.columns if
                  X_tr_fl[cname].dtype in ['int32', 'float32']]
                #   X_tr_fl[cname].dtype in ['int64', 'float64']]

In [ ]:
# Keep selected columns only
# my_cols = categorical_cols + numerical_cols
my_cols =numerical_cols + categorical_cols
X_tr = X_tr_fl[my_cols].copy()
X_val = X_val_fl[my_cols].copy()
# X_test = prop_2016_df[my_cols].copy()

## Optuna + Cross Validation + pipeline + XGB

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from xgboost import XGBRegressor as xgbreg
from sklearn.metrics import mean_absolute_error

def objective(trial, X_tr=X_tr, X_val=X_val, y_tr= y_tr):
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsampele': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014]),
        'max_depth': trial.suggest_categorical('max_depth', [4,5,7,9,15]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'n_estimators': 1000,
        "tree_method": 'gpu_hist',
        "predictor": 'gpu_predictor',
        "gpu_id": 0,
        "verbosity": 1}

    fit_param = {
        'model__eval_set': None,     #[(X_val, y_val)]を変換後に加える。
        'model__eval_metric': 'mae',
        'model__early_stopping_rounds': 500,
        'model__verbose': True}

    ## 前処理
    numerical_transformer = SimpleImputer(strategy='constant') # np.ndarrayの型で返される。 

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    ## Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols),
            ])
    df_tr_ev = pd.concat([X_tr, X_val])
    preprocessor.fit(df_tr_ev)
    X_tr = preprocessor.transform(X_tr)
    X_val = preprocessor.transform(X_val)
    fit_param['model__eval_set'] = [(X_val, y_val)]
    # Define model
    model = xgbreg(objective="reg:squarederror", **param)
    
    # Bundle preprocessing and modeling code in a pipeline
    reg = Pipeline(steps=[('model', model)])

    # cross_validate()は、
    # 評価に複数の指標を考慮できる
    # テストスコアに加えて、学習の時のスコア、学習時間、テストの時間などを算出してくれる。
    scores =  cross_validate(reg,
                             X_tr,
                             y_tr,
                             scoring='neg_mean_absolute_error', #optuna(direction='maximize')にすること！
                             n_jobs=-1,
                             verbose=2,
                             cv=3)
    
    # reg.fit(X_tr, y_tr, **fit_param)
    # y_pred = reg.predict(X_val)
    # mae = mean_absolute_error(y_val, y_pred)
    return scores['test_score'].mean()

In [ ]:
optuna.logging.get_verbosity()

20

In [ ]:
sampler = optuna.samplers.TPESampler(seed=100)
study = optuna.create_study(storage="sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db", 
                            study_name = "study_1", sampler=sampler, pruner=optuna.pruners.MedianPruner(),
                            load_if_exists=False, direction = 'maximize')
study.optimize(objective,n_trials=3,show_progress_bar=True)

[I 2022-04-13 06:23:58,294] A new study created in RDB with name: study_1
/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   21.4s finished


[I 2022-04-13 06:24:21,053] Trial 0 finished with value: -0.06951829294363658 and parameters: {'lambda': 0.14914880124867258, 'alpha': 0.01298606404406187, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 7, 'min_child_weight': 283}. Best is trial 0 with value: -0.06951829294363658.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   13.1s finished


[I 2022-04-13 06:24:35,394] Trial 1 finished with value: -0.06942699352900188 and parameters: {'lambda': 1.864650814036825, 'alpha': 0.022102825780085232, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.012, 'max_depth': 4, 'min_child_weight': 7}. Best is trial 1 with value: -0.06942699352900188.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   40.3s finished


[I 2022-04-13 06:25:16,919] Trial 2 finished with value: -0.07032801955938339 and parameters: {'lambda': 0.0069200034454602885, 'alpha': 0.15091746885099416, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.014, 'max_depth': 15, 'min_child_weight': 284}. Best is trial 1 with value: -0.06942699352900188.


In [ ]:
print(study.best_trial,"\n")
print(study.best_params)
print(study.get_trials())

FrozenTrial(number=1, values=[-0.06942699352900188], datetime_start=datetime.datetime(2022, 4, 13, 6, 24, 21, 71990), datetime_complete=datetime.datetime(2022, 4, 13, 6, 24, 35, 327942), params={'alpha': 0.022102825780085232, 'colsample_bytree': 0.9, 'lambda': 1.864650814036825, 'learning_rate': 0.012, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.7}, distributions={'alpha': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'lambda': LogUniformDistribution(high=10.0, low=0.001), 'learning_rate': CategoricalDistribution(choices=(0.008, 0.01, 0.012, 0.014)), 'max_depth': CategoricalDistribution(choices=(4, 5, 7, 9, 15)), 'min_child_weight': IntUniformDistribution(high=300, low=1, step=1), 'subsample': CategoricalDistribution(choices=(0.6, 0.7, 0.8, 1.0))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None) 

{'alpha': 0.022102825780085232, 'cols

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 保存した結果の読み込み
study = optuna.load_study(study_name='study_1', storage='sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db')

In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,-0.069518,2022-04-13 06:23:58.408941,2022-04-13 06:24:20.981457,0 days 00:00:22.572516,0.012986,0.6,0.149149,0.014,7,283,0.8,COMPLETE
1,1,-0.069427,2022-04-13 06:24:21.071990,2022-04-13 06:24:35.327942,0 days 00:00:14.255952,0.022103,0.9,1.864651,0.012,4,7,0.7,COMPLETE
2,2,-0.070328,2022-04-13 06:24:35.414448,2022-04-13 06:25:16.849533,0 days 00:00:41.435085,0.150917,0.9,0.006920,0.014,15,284,0.7,COMPLETE


In [ ]:
# optunaにcvを含める　optuna単体でハイパラ探索してからcvをかけるのか？

In [ ]:
best_param = study.best_params
best_param

{'alpha': 0.022102825780085232,
 'colsample_bytree': 0.9,
 'lambda': 1.864650814036825,
 'learning_rate': 0.012,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7}

In [ ]:
add_param = {'n_estimators': 1000,
            'tree_method': 'gpu_hist',
            'predictor': 'gpu_predictor',
            'gpu_id': 0,
            'verbosity': 1}

In [ ]:
best_param.update(add_param)
best_param

{'alpha': 0.022102825780085232,
 'colsample_bytree': 0.9,
 'gpu_id': 0,
 'lambda': 1.864650814036825,
 'learning_rate': 0.012,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 1000,
 'predictor': 'gpu_predictor',
 'subsample': 0.7,
 'tree_method': 'gpu_hist',
 'verbosity': 1}

In [ ]:
#final model
# metric や objective の情報は失われてしまうようなので、study.best_params を変数に代入し、metric や objectiveを改めて追加する
model_f = xgbreg(**best_param)

In [ ]:
model_f.predict()

In [ ]:
sample_sub = pd.read_csv("/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/sample_submission.csv", index_col='ParcelId')

In [ ]:
sample_sub

,201610,201611,201612,201710,201711,201712
ParcelId,,,,,,
10754147,0,0,0,0,0,0
10759547,0,0,0,0,0,0
10843547,0,0,0,0,0,0
10859147,0,0,0,0,0,0
10879947,0,0,0,0,0,0
...,...,...,...,...,...,...
168176230,0,0,0,0,0,0
14273630,0,0,0,0,0,0
168040630,0,0,0,0,0,0


# Optuna + Pipeline + XGBReg

In [ ]:
# params = {
#         'max_depth': trial.suggest_int('max_depth', 2, 15),
#         'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
#         'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
# #         'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
# #         'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
#         'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
#         "objective": "reg:linear",
#         "eval_metric": "mae",
#         "tree_method": 'gpu_hist',
#         "predictor": 'gpu_predictor',
#         "gpu_id": 0,
#         "verbosity": 0,
#     }

In [ ]:
# sklearnのpipelineを用いたいならXGBもsklearnAPIで記述する必要がありそう。
# eval_metricおよびobjectiveは、cross_valで最大化or最小化したい評価指標に合わせることが望ましい。
# https://qiita.com/c60evaporator/items/a9a049c3469f6b4872c6

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from xgboost import XGBRegressor as xgbreg
from sklearn.metrics import mean_absolute_error

# # Preprocessing for numerical data
# numerical_transformer = SimpleImputer(strategy='constant')

# # Preprocessing for categorical data
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# # Bundle preprocessing for numerical and categorical data
# preprocessor = ColumnTransformer(transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)])

# # Define model
# model = xgbreg(objective="reg:squarederror")

# # Bundle preprocessing and modeling code in a pipeline
# reg = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('model', model)])

def objective(trial, X_tr=X_tr, X_val=X_val, y_tr= y_tr):
    param = {
        'n_estimators': 10000,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsampele': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014]),
        'max_depth': trial.suggest_categorical('max_depth', [4,5,7,9,15]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        "tree_method": 'gpu_hist',
        "predictor": 'gpu_predictor',
        "gpu_id": 0,
        "verbosity": 0}

    fit_param = {
        'model__eval_set': None,     #[(X_val, y_val)]を変換後に加える。
        'model__eval_metric': 'mae',
        'model__early_stopping_rounds': 500,
        'model__verbose': True}

    numerical_transformer = SimpleImputer(strategy='constant') # np.ndarrayの型で返される。 

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    ## Bundle preprocessing for numerical and categorical data
    # preprocessor = ColumnTransformer(transformers=[
    #         ('num', numerical_transformer, numerical_cols)])
    #         ('cat', categorical_transformer, categorical_cols)]) #元のコード
    
    # X_tr, X_valの全体でpreprocessorを学習させる。後にfit
    preprocessor = ColumnTransformer(transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols),
            ])
    df_tr_ev = pd.concat([X_tr, X_val])
    preprocessor.fit(df_tr_ev)
    X_tr = preprocessor.transform(X_tr)
    X_val = preprocessor.transform(X_val)
    fit_param['model__eval_set'] = [(X_val, y_val)]
    # Define model
    model = xgbreg(objective="reg:squarederror", **param)
    
    # Bundle preprocessing and modeling code in a pipeline
    reg = Pipeline(steps=[('model', model)])
     
    reg.fit(X_tr, y_tr, **fit_param)
    y_pred = reg.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    return mae

In [ ]:
import optuna
sampler = optuna.samplers.TPESampler(seed=100)
study = optuna.create_study(direction = 'minimize', study_name = "study_1", sampler=sampler, pruner=optuna.pruners.MedianPruner())
study.optimize(objective,n_trials=50)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
import optuna


def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    y = trial.suggest_categorical("y", [-1, 0, 1])
    return x ** 2 + y


sampler = optuna.samplers.TPESampler(seed=10)
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=10)

fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [ ]:
for trial in study.get_trials():
    print(f"{trial.number}: {trial.value:.3f} ({trial.params['threshold']})")

In [ ]:
# imputer_cat_pipeline = make_column_transformer(
#     (make_pipeline(SimpleImputer(strategy='constant'), cat_columns_fill_miss),
#     (make_pipeline(SimpleImputer(strategy='most_frequent'), cat_columns_fill_freq),
# )

# encoder_cat_pipeline = make_column_transformer(
#     (OrdinalEncoder(categories=ord_mapping), cat_columns_ord),
#     (OneHotEncoder(), cat_columns_onehot),
# )

# cat_pipeline = Pipeline([
#     ('imp_cat', imputer_cat_pipeline),
#     ('cat_encoder', encoder_cat_pipeline),
# ])


########
# imputer_cat_pipeline = ColumnTransformer([
#         ('imp_miss', SimpleImputer(strategy='constant'), cat_columns_fill_miss),  # fill_value='missing_value' by default
#         ('imp_freq', SimpleImputer(strategy='most_frequent'), cat_columns_fill_freq),
# ])

# encoder_cat_pipeline = ColumnTransformer([
#         ('ordinal', OrdinalEncoder(categories=ord_mapping), cat_columns_ord),
#         ('pass_ord', OneHotEncoder(), cat_columns_onehot),
# ])

# cat_pipeline = Pipeline([
#         ('imp_cat', imputer_cat_pipeline),
#         ('cat_encoder', encoder_cat_pipeline),
# ])

In [ ]:
my_cols =numerical_cols
X_tr = X_tr_fl[my_cols].copy()
X_val = X_val_fl[my_cols].copy()

In [ ]:
# # 中身の動作確認

# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# import xgboost as xgb
# from xgboost import XGBRegressor as xgbreg
# from sklearn.metrics import mean_absolute_error

# param = {
#         'n_estimators': 1000,
#         'tree_method':'gpu_hist',
#         'predictor':"gpu_predictor",
#         'use_label_encoder':False,
#         'random_state':42,
#         'verbosity':2
#         }

# fit_param = {
#     'eval_set': None,
#     'eval_metric': 'mae',
#     'early_stopping_rounds': 50,
#     'verbose': True,}
# # fit_param = {
# #     'model__eval_set': None,
# #     'model__eval_metric': 'mae',
# #     'model__early_stopping_rounds': 50,
# #     'model__verbose': True,}

# numerical_transformer = SimpleImputer(strategy='constant')

# preprocessor = ColumnTransformer(transformers=[
#             ('num', numerical_transformer, numerical_cols),
#             # ('cat', categorical_transformer, categorical_cols),
#             ])
# df_tr_ev = pd.concat([X_tr, X_val])
# preprocessor.fit(df_tr_ev)
# # X_tr_ = pd.DataFrame(preprocessor.transform(X_tr))
# # X_val_ = pd.DataFrame(preprocessor.transform(X_val))
# X_tr_ = preprocessor.transform(X_tr)
# X_val_ = preprocessor.transform(X_val)
# fit_param['eval_set'] = [(X_val_, y_val)]
# # Define model
# reg = xgbreg(objective="reg:squarederror", **param)

# # Bundle preprocessing and modeling code in a pipeline
# # reg = Pipeline(steps=[('preprocessor', preprocessor),
# #                         ('model', model)])
 
# reg.fit(X_tr_, y_tr, **fit_param)
# y_pred = reg.predict(X_val_)
# mae = mean_absolute_error(y_val, y_pred)

In [ ]:
numerical_cols

In [ ]:
# import numpy as np
# import pandas as pd
# import xgboost as xgb
# import gc
# import sqlite3

# print('Loading data ...')

# train = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/train_2016_v2.csv')
# prop = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/properties_2016.csv')
# sample = pd.read_csv('/content/gdrive/MyDrive/kaggle/zillow_prize/datasets/sample_submission.csv')

# print('Binding to float32')

Loading data ...


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Binding to float32


# Optuna + XGB.CV

In [ ]:
# for c, dtype in zip(prop.columns, prop.dtypes):
# 	if dtype == np.float64:
# 		prop[c] = prop[c].astype(np.float32)

# print('Creating training set ...')

# df_train = train.merge(prop, how='left', on='parcelid')

# x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
# y_train = df_train['logerror'].values
# print(x_train.shape, y_train.shape)

Creating training set ...
(90275, 55) (90275,)


In [ ]:
# train_columns = x_train.columns

# for c in x_train.dtypes[x_train.dtypes == object].index.values:
#     x_train[c] = (x_train[c] == True)

# del df_train; gc.collect()

172

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, train_size=0.8, test_size=0.2)
# split = 80000
# x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

print('Building DMatrix...')

d_tr = xgb.DMatrix(x_tr, label=y_tr)
d_val = xgb.DMatrix(x_val, label=y_val)

del x_tr, x_val; gc.collect()

print('Training ...')

Building DMatrix...
Training ...


In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split,KFold,cross_validate

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
#         'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
#         'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "objective": "reg:linear",
        "eval_metric": "mae",
        "tree_method": 'gpu_hist',
        "predictor": 'gpu_predictor',
        "gpu_id": 0,
        "verbosity": 0,
    }
    
#     model = xgb.train(param_sch,
#                       dtrain=d_tr,
#                       num_boost_round=10,
#                       evals=[(d_val, 'eval')],
#                       evals_result={},
#                       verbose_eval=10,
#                       early_stopping_rounds=100,
#                       )
#     kf = KFold(n_splits=3, shuffle=True, random_state=0)
#     scores = cross_validate(model, X=, y=y_train,scoring='mae',cv=kf,n_jobs=-1)

    # xgb.trainではval_setを指定する必要があるがxgb.cvではcvする際にval_setを自動で各cvごとに設定してくれる
    cv_results = xgb.cv(params=params,
                        dtrain=d_tr,
                        num_boost_round=10000,
                        nfold=3,
                        verbose_eval=10,
                        early_stopping_rounds=500)
    return cv_results["test-mae-mean"].min()




# params = {}
# params['eta'] = 0.02
# params['objective'] = 'reg:linear'
# params['eval_metric'] = 'mae'
# params['max_depth'] = 4
# params['silent'] = 1

# watchlist = [(d_train, 'train'), (d_valid, 'valid')]
# clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

In [ ]:
study = optuna.create_study(direction='minimize',
                            study_name="study_1",
                            storage=f"sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db",
                            load_if_exists=True,
                            pruner=optuna.pruners.PercentilePruner(60.0))

study.optimize(objective, n_trials=50)

In [ ]:
# 保存した結果の読み込み
study = optuna.load_study(study_name='study_1', storage='sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db')

In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_eta,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,2022-04-07 05:10:37.831319,NaT,NaT,0.248701,0.10,12,17,0.95,RUNNING
1,1,0.068626,2022-04-07 05:10:58.809998,2022-04-07 05:11:21.330014,0 days 00:00:22.520016,0.165255,0.01,7,16,0.95,COMPLETE
2,2,0.070715,2022-04-07 05:11:21.405142,2022-04-07 05:11:55.750425,0 days 00:00:34.345283,0.626758,0.06,11,4,0.60,COMPLETE
3,3,0.071048,2022-04-07 06:38:32.409541,2022-04-07 06:39:12.745684,0 days 00:00:40.336143,0.677216,0.07,15,18,0.75,COMPLETE
4,4,0.068714,2022-04-07 06:39:12.815621,2022-04-07 06:39:22.567490,0 days 00:00:09.751869,0.706567,0.05,5,12,0.90,COMPLETE
5,5,0.069672,2022-04-07 06:39:22.631557,2022-04-07 06:40:03.612910,0 days 00:00:40.981353,0.855272,0.01,9,2,0.90,COMPLETE
6,6,0.069478,2022-04-07 06:40:03.683574,2022-04-07 06:40:24.408330,0 days 00:00:20.724756,0.224880,0.04,9,4,0.60,COMPLETE
7,7,0.068532,2022-04-07 06:40:24.493149,2022-04-07 06:40:33.395288,0 days 00:00:08.902139,0.239044,0.05,4,9,0.70,COMPLETE
8,8,0.069670,2022-04-07 06:40:33.464726,2022-04-07 06:41:19.314417,0 days 00:00:45.849691,0.491400,0.01,11,8,0.95,COMPLETE
9,9,0.069179,2022-04-07 06:41:19.380544,2022-04-07 06:41:29.475578,0 days 00:00:10.095034,0.648173,0.09,6,13,0.85,COMPLETE


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
# テストデータでの推測の準備

param_f = study.best_params
param_f["objective"] = "reg:linear"
param_f["eval_metric"] = "mae"
param_f["predictor"] = 'gpu_predictor'
param_f["gpu_id"] = 0
param_f["verbosity"] = 0

In [ ]:
# evals = [(xgb_train, ‘train’), (xgb_eval, ‘eval’)]

In [ ]:
# https://datatechlog.com/how_to_use_xgboost_and_optuna_for_parameter_optimization/#toc26
model = xgb.train(params=param_f,
                  dtrain=d_tr,
                  num_boost_round=10000,
                  early_stopping_rounds=500,
                  evals=[(d_tr, 'train'), (d_val, 'eval')],
                  evals_result={},
                  verbose_eval=10,
                 )
                  

[0]	train-mae:0.472983	eval-mae:0.472478
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 500 rounds.
[10]	train-mae:0.2921	eval-mae:0.291605
[20]	train-mae:0.187704	eval-mae:0.187195
[30]	train-mae:0.128984	eval-mae:0.128389
[40]	train-mae:0.097369	eval-mae:0.096796
[50]	train-mae:0.081447	eval-mae:0.08097
[60]	train-mae:0.074105	eval-mae:0.073743
[70]	train-mae:0.070865	eval-mae:0.070537
[80]	train-mae:0.069432	eval-mae:0.06912
[90]	train-mae:0.068792	eval-mae:0.068473
[100]	train-mae:0.068473	eval-mae:0.068146
[110]	train-mae:0.068315	eval-mae:0.067989
[120]	train-mae:0.068204	eval-mae:0.067877
[130]	train-mae:0.06815	eval-mae:0.06784
[140]	train-mae:0.068121	eval-mae:0.067815
[150]	train-mae:0.068094	eval-mae:0.067802
[160]	train-mae:0.068078	eval-mae:0.067794
[170]	train-mae:0.068071	eval-mae:0.0678
[180]	train-mae:0.068064	eval-mae:0.067798
[190]	train-mae:0.068042	eval-mae:0.067782
[200]	train-mae:0

In [ ]:
# https://colab.research.google.com/drive/1UDHCMgX-_953h1Vc_kAkjQghe2foVT3H これもみる

In [ ]:
print('Building test set ...')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

Building test set ...


In [ ]:
x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2

In [ ]:
print('Predicting on test ...')

p_test = model.predict(d_test)

Predicting on test ...


In [ ]:
d_tr.num_row()

72220

In [ ]:
p_test.shape

(2985217,)

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_obs, y_pred)

AttributeError: ignored

In [ ]:
p_test.shape

(2985217,)

In [ ]:
# import sqlite3
# con = sqlite3.connect('/content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db')
# cur = con.cursor()

In [ ]:
# cur.execute('select * from study_1')
# for row in cur:
#     # rowオブジェクトでデータが取得できる。タプル型の結果が取得
#     print(row)

In [ ]:
# from sqlalchemy import create_engine
# import sqlalchemy
# engine = create_engine('sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db', echo = True)


In [ ]:
# from sqlalchemy.orm import sessionmaker
# Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)
# session = Session() 

In [ ]:
print(engine)

Engine(sqlite:////content/gdrive/MyDrive/kaggle/zillow_prize/results/study_1.db)


In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))